In [1]:
# Import Libraries
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Conv2D,Flatten,Dropout,MaxPooling2D 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers, Model

import os 
import numpy as np 
import matplotlib.pyplot as plt 

In [2]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip' 

path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True) 

PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered') 

68616192/68606236 [==============================] - 0s 0us/step


In [3]:
PATH

'/root/.keras/datasets/cats_and_dogs_filtered'

In [4]:
! ls $datapath

drive  sample_data


In [ ]:
#path_to_zip = "/content/drive/MyDrive/Deep_Learning/Transfer_Learning/"
#PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs.zip') 
#New_PATH="/content/drive/MyDrive/Deep_Learning/Transfer_Learning/cats_and_dogs" 

#!unzip $PATH -d $New_PATH 

In [5]:
datapath = "/content/drive/MyDrive/Deep_Learning/Transfer_Learning/cats_and_dogs/cats_and_dogs_filtered"
datapath

'/content/drive/MyDrive/Deep_Learning/Transfer_Learning/cats_and_dogs/cats_and_dogs_filtered'

In [6]:
! ls $datapath

train  validation  vectorize.py


In [7]:
train_dir = os.path.join(PATH,'train') 
validation_dir = os.path.join(PATH,'validation') 

In [8]:
train_dir

'/root/.keras/datasets/cats_and_dogs_filtered/train'

In [9]:
train_cats_dir = os.path.join(train_dir,'cats') # Directory with our training cat pics 
train_dogs_dir = os.path.join(train_dir,'dogs') # Directory with our training dogs pics 
validation_cats_dir = os.path.join(validation_dir,'cats') # Directory with our validation cats pics 
validation_dogs_dir = os.path.join(validation_dir,'dogs') # Directory with our validation dogs pics 

In [10]:
num_cat_tr = len(os.listdir(train_cats_dir))
num_dog_tr = len(os.listdir(train_dogs_dir))
num_cat_val = len(os.listdir(validation_cats_dir))
num_dog_val = len(os.listdir(validation_dogs_dir))

total_train = num_cat_tr + num_dog_tr 
total_val = num_cat_val + num_dog_val 

In [11]:
print(f"Total trainning cat images  : {num_cat_tr}")
print(f"Total trainning dog images  : {num_dog_tr}")
print(f"Total validation cat images : {num_cat_val}")
print(f"Total validation dog images : {num_dog_val}")
print("----------------------------------------------") 
print(f"Total training images   : {total_train}") 
print(f"Total validation images : {total_val}") 



Total trainning cat images  : 1000
Total trainning dog images  : 1000
Total validation cat images : 500
Total validation dog images : 500
----------------------------------------------
Total training images   : 2000
Total validation images : 1000


In [12]:
batch_size = 128  # Power of 2 ---> 64, 128, 256, 512
epochs = 30 
IMG_HEIGHT = 224 
IMG_WIDTH = 224

# VGG16

# 1.Image Augmentation

In [13]:
train_image_generator = ImageDataGenerator(rescale=1./255, 
                                           rotation_range=45, 
                                           horizontal_flip=True, 
                                           zoom_range=0.5, 
                                           width_shift_range=.15, 
                                           height_shift_range=0.15) # generator for our training data 
validation_image_generator = ImageDataGenerator(rescale=1./255) # generator for our validation data 

# 2.Trainning and Validation sets




In [14]:
train_data_gen= train_image_generator.flow_from_directory( batch_size=batch_size, 
                                                     directory = train_dir, shuffle = True, 
                                                     target_size=(IMG_HEIGHT,IMG_WIDTH), 
                                                     class_mode='binary') # Multiclass ---> class_mode ="caterogry" 

Found 2000 images belonging to 2 classes.


In [15]:
validation_data_gen = validation_image_generator.flow_from_directory( batch_size=batch_size, 
                                                         directory = validation_dir, 
                                                         shuffle = True, target_size=(IMG_HEIGHT,IMG_WIDTH), 
                                                         class_mode='binary') # Multiclass ---> class_mode ="categorical" 

Found 1000 images belonging to 2 classes.


# 3.Loading the base Model

In [16]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [17]:
# We dont have to train all layers,we make them non trainnable
for layer in base_model.layers:
  layer.trainable=False  #--->weight should not be updated in BackPropagation

# 4.Compile and Fit

In [18]:
x = layers.Flatten()(base_model.output)  # Flatten the output to 1 dimension

# Add a FC layer with 512 hidden units and ReLu Act 
x = layers.Dense(512,activation='relu')(x) 

# dropout rate 0.5 
x = layers.Dropout(0.5)(x) 

# Adding final layer
x = layers.Dense(1,activation='sigmoid')(x) # 1---> Binary class   2 or 3----> Multiclass (softmax)

model = Model(base_model.input,x)

#model.compile()

model.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])  #Optimizers--> RMSprop, Adam, SGD
 
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)


In [19]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [22]:
! pip install tensorboardcolab

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for tensorboardcolab: filename=tensorboardcolab-0.0.22-py3-none-any.whl size=3859 sha256=b7a29f568638c2eb22a2c915db4c4098a24e4ed7a7b5d76f5214d99c6f5f4211
  Stored in directory: /root/.cache/pip/wheels/69/4e/4a/1c6c267395cb10edded1050df12af165d3254cfce324e80941
Successfully built tensorboardcolab


In [24]:
import datetime, os
datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

'20221006-111617'

In [20]:
import datetime, os

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S")) 

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

vgg16_model = model.fit(train_data_gen, validation_data=validation_data_gen,steps_per_epoch = 10, epochs = 10,callbacks=[tensorboard_callback]) #callbacks=[tensorboard_callback]
 

Epoch 1/10
10/10 [==============================] - 59s 4s/step - loss: 1.4800 - acc: 0.5341 - val_loss: 0.6550 - val_acc: 0.5690
Epoch 2/10
10/10 [==============================] - 27s 3s/step - loss: 0.6547 - acc: 0.6550 - val_loss: 0.7516 - val_acc: 0.5840
Epoch 3/10
10/10 [==============================] - 30s 3s/step - loss: 0.6647 - acc: 0.6672 - val_loss: 0.3860 - val_acc: 0.8390
Epoch 4/10
10/10 [==============================] - 26s 3s/step - loss: 0.6128 - acc: 0.6727 - val_loss: 0.3743 - val_acc: 0.8470
Epoch 5/10
10/10 [==============================] - 26s 3s/step - loss: 0.5701 - acc: 0.7062 - val_loss: 0.3613 - val_acc: 0.8380
Epoch 6/10
10/10 [==============================] - 27s 3s/step - loss: 0.5911 - acc: 0.7005 - val_loss: 0.3744 - val_acc: 0.8260
Epoch 7/10
10/10 [==============================] - 29s 3s/step - loss: 0.5451 - acc: 0.7180 - val_loss: 0.5651 - val_acc: 0.6760
Epoch 8/10
10/10 [==============================] - 29s 3s/step - loss: 0.5382 - acc: 0.72

In [37]:
!kill 652

In [ ]:
#vgg16_model = model.fit(train_data_gen, validation_data = validation_data_gen, steps_per_epoch = 10, epochs = 100)

In [38]:
%reload_ext tensorboard
%tensorboard --logdir logs 


<IPython.core.display.Javascript object>